# Data Preprocessing

The next step is to

In [1]:
#@title Imports for Project
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../data/rideshare_kaggle.csv')
df.head()

: 

: 

Data Cleaning

In [5]:
#Identify number of null values per column in the database
df.isna().sum()

id                                 0
timestamp                          0
hour                               0
day                                0
month                              0
datetime                           0
timezone                           0
source                             0
destination                        0
cab_type                           0
product_id                         0
name                               0
price                          55095
distance                           0
surge_multiplier                   0
latitude                           0
longitude                          0
temperature                        0
apparentTemperature                0
short_summary                      0
long_summary                       0
precipIntensity                    0
precipProbability                  0
humidity                           0
windSpeed                          0
windGust                           0
windGustTime                       0
v

In [6]:
#Drop rows that have a null value in the price column
df = df[df['price'].notna()]

In [ ]:
#Remove duplicates in the ID column
df = df.drop_duplicates('id')

In [11]:
# Check for duplicate rows
duplicates = df.duplicated()
duplicates_num = duplicates.sum()

print(duplicates_num)

0


In [ ]:
# Removing outliers in the price, ...

In [12]:
# Reduce data to only contain longitude between -180 and 180, and latitude between -90 and 90
df = df[(df['longitude'] > -180) & (df['longitude'] < 180)]
df = df[(df['latitude'] > -90) & (df['latitude'] < 90)]

In [1]:
# Separate the Datetime column into time, year, month, date and day of the week

df['datetime'] = pd.to_datetime(df['datetime'])

df['time'] = df['datetime'].dt.strftime('%H:%M')
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.strftime('%B')
df['day'] = df['datetime'].dt.day
df['day_of_week'] = df['datetime'].dt.dayofweek 

day_names = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
df['day_of_week'] = df['day_of_week'].map(day_names)

df.head()

NameError: name 'pd' is not defined